# Using Pandas Library to Clean Data

In [1]:
import pandas as pd

## Data Import/Ingestion

In [2]:
#Loading first dataframe (total performance)
performance_inclusion = pd.read_excel('C:/Users/mayow/OneDrive/My Projects [data analytics]/Telecommunications Site Outages Analysis and Reporting/assets/datasets/performance_and_outage_report.xlsx', sheet_name='Incl Exclusion')
performance_inclusion.columns = performance_inclusion.iloc[0]
performance_inclusion = performance_inclusion[1:].reset_index(drop=True)


#Loading second dataframe (performance excluding external factors)
performance_exclusion = pd.read_excel('C:/Users/mayow/OneDrive/My Projects [data analytics]/Telecommunications Site Outages Analysis and Reporting/assets/datasets/performance_and_outage_report.xlsx', sheet_name='Excl Exclusion')
performance_exclusion.columns = performance_exclusion.iloc[0]
performance_exclusion = performance_exclusion[1:].reset_index(drop=True)

#Loading third dataframe (outages)
outages = pd.read_excel('C:/Users/mayow/Downloads/Work - Offline/Performance - Lookup File/IHS Daily Performance.xlsx', sheet_name='Power TTs')

In [8]:
print(performance_inclusion.head(0))
print('*******************************')
print(performance_exclusion.head(0))
print('*******************************')
print(outages.head(0))

#Check if data is properly loaded, especially the headers

Empty DataFrame
Columns: [S/N, IHS Site ID, Site ID, MS/Colo Sites, Region IHS, Region state, REGION FOCUSED, State, Cluster, Regional Supervisor, Regional coordinator , Priority, Significant Priority class on site - All Customers, January Counts of PA<99.99, January Average Availability, 2023-01-01 00:00:00, 2023-01-02 00:00:00, 2023-01-03 00:00:00, 2023-01-04 00:00:00, 2023-01-05 00:00:00, 2023-01-06 00:00:00, 2023-01-07 00:00:00, 2023-01-08 00:00:00, 2023-01-09 00:00:00, 2023-01-10 00:00:00, 2023-01-11 00:00:00, 2023-01-12 00:00:00, 2023-01-13 00:00:00, 2023-01-14 00:00:00, 2023-01-15 00:00:00, 2023-01-16 00:00:00, 2023-01-17 00:00:00, 2023-01-18 00:00:00, 2023-01-19 00:00:00, 2023-01-20 00:00:00, 2023-01-21 00:00:00, 2023-01-22 00:00:00, 2023-01-23 00:00:00, 2023-01-24 00:00:00, 2023-01-25 00:00:00, 2023-01-26 00:00:00, 2023-01-27 00:00:00, 2023-01-28 00:00:00, 2023-01-29 00:00:00, 2023-01-30 00:00:00, 2023-01-31 00:00:00, Feburary  Counts of PA<99.99, Feburary Average Availability

#### Data Cleaning for Performance

In [28]:
# PERFORMANCE CLEANING
# For the performance dataframes, we need to keep just the site_id columns and the columns with performance [the columns with performance have dates as the column headers]
# so what we would do is to create a function that checks if column header is a date

# Function to check if a column name is a date
def is_date(string):
    try:
        pd.to_datetime(string)
        return True
    except ValueError:
        return False
    

# Create a variable that stores found columns
date_columns_inc = [col for col in performance_inclusion.columns if is_date(col)]
date_columns_exc = [col for col in performance_exclusion.columns if is_date(col)]

# Add the above columns to the 'IHS Site ID' column
columns_to_keep_inc = ["IHS Site ID"] + date_columns_inc
columns_to_keep_exc = ["IHS Site ID"] + date_columns_exc

# Create new dataframes with 'columns to keep'
inc = performance_inclusion[columns_to_keep_inc]
exc = performance_exclusion[columns_to_keep_exc]

In [30]:
exc.head(2)

,IHS Site ID,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00,2023-01-06 00:00:00,2023-01-07 00:00:00,2023-01-08 00:00:00,2023-01-09 00:00:00,...,2023-12-22 00:00:00,2023-12-23 00:00:00,2023-12-24 00:00:00,2023-12-25 00:00:00,2023-12-26 00:00:00,2023-12-27 00:00:00,2023-12-28 00:00:00,2023-12-29 00:00:00,2023-12-30 00:00:00,2023-12-31 00:00:00
0,IHS_ABE_0023B,100,100,100,100,100,100,100,100,100,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,IHS_ABJ_0103B,100,100,100,100,100,100,91.688657,100,95.979167,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [31]:
#PERFORMANCE CLEANING contd

# We have out dataset with Site_IDs on first column and daily performances on subsequent columns, but we want to unpivot these performance from the site ID
# This will enable us have just 3 columns with Site_IDs, Date, Performance
# we will be using the melt method on pandas

inc = pd.melt(inc, id_vars=['IHS Site ID'], var_name='Date', value_name='Performance')
exc = pd.melt(exc, id_vars=['IHS Site ID'], var_name='Date', value_name='Performance')

In [32]:
exc.tail(2)

# Some performance fields are empty

,IHS Site ID,Date,Performance
479824,IHS_KAN_1380B,2023-12-31 00:00:00,NaT
479825,IHS_ZAM_0020B,2023-12-31 00:00:00,NaT


In [33]:
# We also need to remove columns where performance is not available in our dataframe
inc = inc.dropna(subset=['Performance'])
exc = exc.dropna(subset=['Performance'])

# And an extra column named 'Exclusions' to specify or diffrentiate inc and exc because we are going to merge data into one
inc['Exclusions'] = 'Including'
exc['Exclusions'] = 'Excluding'

In [34]:
exc.tail(2)

#Testing out df [We notices date is year 2023 instead of 2024]

,IHS Site ID,Date,Performance,Exclusions
221557,IHS_KAN_1380B,2023-06-17 00:00:00,100,Excluding
221558,IHS_ZAM_0020B,2023-06-17 00:00:00,100,Excluding


In [35]:
# New dataframe is created from the merged dfs (inc and exc)
performance = pd.concat([inc, exc],ignore_index=True)

# Date Column is properly formated as a date
performance['Date'] = pd.to_datetime(performance['Date'])

# Some dates where mistakenly entered as 2023 instead of 2024, but since we know our data includes in 2024
# We can handle that by adding a year to years in the df
# Locate where we have 2023 and add 1
performance.loc[performance['Date'].dt.year == 2023, 'Date'] += pd.DateOffset(years=1)

In [36]:
performance.tail(2)

,IHS Site ID,Date,Performance,Exclusions
360011,IHS_KAN_1380B,2024-06-17,100,Excluding
360012,IHS_ZAM_0020B,2024-06-17,100,Excluding


#### Data Cleaning for Outages

In [38]:
# OUTAGES CLEANING
# For the otages dataframes, select needed columns

outages = outages[['IHS Site ID',
       'Process No', 'Task No.', 'Fault MTTR', 'Time Fault occured',
       'Time Site on Air', 'Source of Escalation', 'Impact',
       'Root Cause Analysis', 'Root Cause Type', 'Resolution', 'Diesel Level','Exclusion (Y/N)', 'Day', 'Week', 'Ghr', 'Mins', 'class']]

In [39]:
# The derive_rca function is designed to classify the root cause of an issue based on the content of the Root Cause Analysis column in a dataframe. 
# It returns a categorized string based on keywords found in the analysis text.

def derive_rca(row):
    if pd.isnull(row['Root Cause Analysis']):
        return 'Unknown'  # Handle NaN values appropriately
    
    rca = row['Root Cause Analysis'].lower()
    
    if 'access' in rca:
        return 'Access'
    elif 'fuel' in rca or 'diesel' in rca or 'separator' in rca:
        return 'Diesel Related'
    elif 'hybrid' in rca:
        return 'Hybrid issues'
    elif 'compression' in rca:
        return 'Loss Compression'
    elif 'servicing' in rca or 'plan' in rca or 'activi' in rca:
        return 'Planned activity'
    elif 'grid' in rca:
        return 'Grid'
    elif 'rectif' in rca:
        return 'Rectifier'
    elif 'theft' in rca or 'vanda' in rca:
        return 'Theft/Vandalism'
    elif 'dg' in rca or 'cable' in rca:
        return 'Spares Related'
    elif 'ats' in rca:
        return 'ATS'
    elif 'breaker' in rca:
        return 'Breaker'
    else:
        return 'Other Issues'

In [40]:
# Convert 'Fault MTTR' from seconds to 'HH:MM:SS' format if it's numeric
outages['Fault MTTR'] = outages['Fault MTTR'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%H:%M:%S') if isinstance(x, (int, float)) else x)

# Convert 'Time Fault occured' to datetime
outages['Time Fault occured'] = pd.to_datetime(outages['Time Fault occured'])

# Convert 'Time Site on Air' to datetime
outages['Time Site on Air'] = pd.to_datetime(outages['Time Site on Air'])

# Calculate the difference between 'Time Site on Air' and 'Time Fault occured'
outages['confirm MTTR'] = outages['Time Site on Air'] - outages['Time Fault occured']

# Convert the time difference to string format
outages['confirm MTTR'] = outages['confirm MTTR'].apply(lambda x: str(x))

# Convert the time difference string back to timedelta
outages['confirm MTTR'] = pd.to_timedelta(outages['confirm MTTR'])

# Extract date part from 'Day'
outages['Day'] = pd.to_datetime(outages['Day']).dt.date

# Define a threshold of 1 hour 59 minutes
threshold = pd.Timedelta(hours=1, minutes=59)

# Determine if 'confirm MTTR' exceeds the threshold
outages['SLA Status'] = outages['confirm MTTR'].apply(lambda x: 'Breached' if x > threshold else 'Achieved')

# Apply the derive_rca function to each row to derive 'RCA [Derived]'
outages['RCA [Derived]'] = outages.apply(derive_rca, axis=1)

In [41]:
outages.tail(3)

,IHS Site ID,Process No,Task No.,Fault MTTR,Time Fault occured,Time Site on Air,Source of Escalation,Impact,Root Cause Analysis,Root Cause Type,...,Diesel Level,Exclusion (Y/N),Day,Week,Ghr,Mins,class,confirm MTTR,SLA Status,RCA [Derived]
20883,IHS_ABJ_1057A,0,NOCINC0002416315,00:35:04,2024-06-17 23:24:55,2024-06-17 23:59:59,0,Site Down,Rectifier Module,Rectifier,...,0,N,2024-06-17,Week 25,0,35.066667,NaN,0 days 00:35:04,Achieved,Rectifier
20884,IHS_PLA_0708E,0,NOCINC0002413718,00:15:22,2024-06-17 21:16:57,2024-06-17 21:32:19,0,Site Down,National Grid - Grid Outage Issue,Grid,...,0,N,2024-06-17,Week 25,0,15.366667,NaN,0 days 00:15:22,Achieved,Grid
20885,IHS_ADA_0913B,0,NOCINC0002413892,00:45:31,2024-06-17 16:29:36,2024-06-17 17:15:07,0,Site Down,Hybrid Software,Hybrid,...,0,N,2024-06-17,Week 25,0,45.516667,NaN,0 days 00:45:31,Achieved,Hybrid issues


## Data Cleaned, Export to CSV

In [43]:
performance.to_csv('C:/Users/mayow/OneDrive/My Projects [data analytics]/Telecommunications Site Outages Analysis and Reporting/assets/datasets/performance.csv', index=False)
outages.to_csv('C:/Users/mayow/OneDrive/My Projects [data analytics]/Telecommunications Site Outages Analysis and Reporting/assets/datasets/outages.csv', index=False)